数据集链接：https://www.kaggle.com/datasets/thedevastator/chinese-medical-dialogue-model

In [1]:
import chardet
import glob
import os
from tqdm import tqdm

def detect_file_encoding(file_path):
    """检测文件的编码格式"""
    # 读取文件的一部分来检测编码（避免大文件读取全部内容）
    with open(file_path, 'rb') as file:
        # 读取前10000个字节进行检测
        raw_data = file.read(10000)
        result = chardet.detect(raw_data)
        
    return {
        'file_name': os.path.basename(file_path),
        'encoding': result['encoding'],
        'confidence': result['confidence']
    }

def main():
    # 获取所有CSV文件
    csv_files = glob.glob('D:/Desktop/finetune/archive/*.csv')
    
    print("开始检测文件编码...")
    
    # 遍历检测每个CSV文件
    for csv_file in tqdm(csv_files, desc="检测文件编码"):
        try:
            result = detect_file_encoding(csv_file)
            print(f"\n文件名: {result['file_name']}")
            print(f"编码格式: {result['encoding']}")
            print(f"置信度: {result['confidence']:.2%}")
            print("-" * 50)
            
        except Exception as e:
            print(f"处理文件 {csv_file} 时出错: {str(e)}")

main()

开始检测文件编码...


检测文件编码: 100%|██████████| 9/9 [00:00<00:00, 91.99it/s]


文件名: finetune_test.csv
编码格式: None
置信度: 0.00%
--------------------------------------------------

文件名: finetune_train.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: finetune_validation.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: pretrain_test.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: pretrain_train.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: pretrain_validation.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: reward_test.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: reward_train.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------

文件名: reward_validation.csv
编码格式: utf-8
置信度: 99.00%
--------------------------------------------------


In [5]:
import pandas as pd
csv_file = 'D:/Desktop/finetune/archive/finetune_train.csv'
df = pd.read_csv(csv_file)
df_cleaned = df.drop('input', axis=1)
df_cleaned.to_csv('D:/Desktop/finetune/archive/finetune_train.csv', index=False)
print(df_cleaned.head())

C:\Users\13686\AppData\Local\Temp\ipykernel_23664\145930813.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


          instruction                                             output
0         血热的临床表现是什么?  初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，...
1  帕金森叠加综合征的辅助治疗有些什么？                       综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗
2    卵巢癌肉瘤的影像学检查有些什么？                       超声漏诊；声像图；MR检查；肿物超声；术前超声；CT检查
3        婴儿耳朵湿疹流水怎么治疗  婴儿湿疹又称“奶癣”，是常见的一种过敏性皮肤病，多在生后2－3个月发病，1岁以后逐渐好转。湿...
4      低T3综合征的并发症是什么？                               心力衰竭；甲状腺结节；糖尿病；感染性休克


In [6]:
from datasets import load_dataset, load_from_disk, DatasetDict
import pandas as pd
import glob
import os
from tqdm import tqdm

# 创建输出目录
output_base_dir = 'D:/Desktop/NLP/finetune/archive_arrow'
os.makedirs(output_base_dir, exist_ok=True)

# 获取所有CSV文件
csv_files = glob.glob(r'D:/Desktop/NLP/finetune/archive/*.csv')

# 添加调试信息
print(f"CSV文件列表: {csv_files}")
print(f"输出目录: {output_base_dir}")

# 遍历处理每个CSV文件
for csv_file in tqdm(csv_files, desc="转换CSV文件"):
    try:
        # 获取文件名（不含扩展名）
        file_name = os.path.splitext(os.path.basename(csv_file))[0]
        
        if file_name in ['reward_train']:
            print(f"处理文件: {csv_file}")
            
            # 设置输出路径
            output_path = os.path.join(output_base_dir, file_name)
            
            # 使用pandas读取CSV文件并显示信息
            df = pd.read_csv(csv_file)
            print(f"数据行数: {len(df)}")
            
            # 将清理后的数据转换为datasets格式
            # 修改这里：使用正确的load_dataset参数
            dataset = load_dataset('csv', data_files={'train': csv_file})
            
            # 保存为arrow格式
            dataset.save_to_disk(output_path)
            
            # 验证文件是否成功保存
            if os.path.exists(output_path):
                print(f"文件已成功保存到: {output_path}")
                
                # 尝试加载保存的数据集
                loaded_dataset = load_from_disk(output_path)
                print(f"成功加载数据集，列名: {loaded_dataset.column_names}")
            else:
                print(f"保存失败，文件不存在: {output_path}")
            
            print(f"成功转换: {file_name}")
        
    except Exception as e:
        print(f"处理文件 {file_name} 时出错: {str(e)}")

CSV文件列表: ['D:/Desktop/NLP/finetune/archive\\finetune_test.csv', 'D:/Desktop/NLP/finetune/archive\\finetune_train.csv', 'D:/Desktop/NLP/finetune/archive\\finetune_validation.csv', 'D:/Desktop/NLP/finetune/archive\\pretrain_test.csv', 'D:/Desktop/NLP/finetune/archive\\pretrain_train.csv', 'D:/Desktop/NLP/finetune/archive\\pretrain_validation.csv', 'D:/Desktop/NLP/finetune/archive\\reward_test.csv', 'D:/Desktop/NLP/finetune/archive\\reward_train.csv', 'D:/Desktop/NLP/finetune/archive\\reward_validation.csv']
输出目录: D:/Desktop/NLP/finetune/archive_arrow


转换CSV文件:   0%|          | 0/9 [00:00<?, ?it/s]

处理文件: D:/Desktop/NLP/finetune/archive\reward_train.csv
数据行数: 3800


Saving the dataset (0/1 shards):   0%|          | 0/3800 [00:00<?, ? examples/s]

转换CSV文件: 100%|██████████| 9/9 [00:02<00:00,  3.90it/s]

文件已成功保存到: D:/Desktop/NLP/finetune/archive_arrow\reward_train
成功加载数据集，列名: {'train': ['question', 'response_chosen', 'response_rejected']}
成功转换: reward_train


In [10]:
from datasets import load_dataset
import glob
import os
from tqdm import tqdm

# 创建输出目录
output_base_dir = 'D:/Desktop/NLP/finetune/archive_arrow'
os.makedirs(output_base_dir, exist_ok=True)

# 获取所有CSV文件
csv_files = glob.glob('D:/Desktop/finetune/archive/*.csv')

# 遍历处理每个CSV文件
for csv_file in tqdm(csv_files, desc="转换CSV文件"):
    try:
        # 获取文件名（不含扩展名）
        file_name = os.path.splitext(os.path.basename(csv_file))[0]
        
        # 设置输出路径
        output_path = os.path.join(output_base_dir, file_name)
        
        # 加载并转换单个CSV文件
        dataset = load_dataset('csv', data_files=csv_file)
        
        # 保存为arrow格式
        dataset.save_to_disk(output_path)
        
        print(f"成功转换: {file_name}")
        
        
    except Exception as e:
        print(f"处理文件 {file_name} 时出错: {str(e)}")

转换CSV文件:   0%|          | 0/9 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

转换CSV文件:  11%|█         | 1/9 [00:01<00:08,  1.02s/it]

处理文件 finetune_test 时出错: An error occurred while generating the dataset


Saving the dataset (0/3 shards):   0%|          | 0/2066589 [00:00<?, ? examples/s]

...

转换CSV文件:  89%|████████▉ | 8/9 [00:04<00:00,  2.59it/s]

成功转换: reward_train


Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

转换CSV文件: 100%|██████████| 9/9 [00:04<00:00,  1.89it/s]

成功转换: reward_validation


In [11]:
from datasets import load_from_disk

# 加载数据集
dataset_path = "D:/Desktop/finetune/archive_arrow/finetune_train"  # 替换为你的路径
dataset = load_from_disk(dataset_path)

print(dataset.column_names)
# 显示基本信息
print(f"数据集格式: {dataset}")
print(f"\n数据集大小: {len(dataset['train'])} 条")
print(f"\n数据集特征: {dataset['train'].features}")

# 显示前几条数据
print("\n数据样例:")
print(dataset['train'][:3])

# 显示每个列的数据类型和非空值数量
print("\n列统计信息:")
for column in dataset['train'].features:
    non_null_count = sum(1 for x in dataset['train'][column] if x is not None)
    print(f"列名: {column}")
    print(f"数据类型: {dataset['train'].features[column]}")
    print(f"非空值数量: {non_null_count}")
    print("---")

{'train': ['instruction', 'output']}
数据集格式: DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 2066589
    })
})

数据集大小: 2066589 条

数据集特征: {'instruction': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}

数据样例:
{'instruction': ['血热的临床表现是什么?', '帕金森叠加综合征的辅助治疗有些什么？', '卵巢癌肉瘤的影像学检查有些什么？'], 'output': ['初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。', '综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗', '超声漏诊；声像图；MR检查；肿物超声；术前超声；CT检查']}

列统计信息:
列名: instruction
数据类型: Value(dtype='string', id=None)
非空值数量: 2066589
---
列名: output
数据类型: Value(dtype='string', id=None)
非空值数量: 2066588
---
